<a href="https://colab.research.google.com/github/DeokwonWang/deeplearnigstudy/blob/main/07_XOR_backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

# 환경변수 지정
# 입력값, 타겟값
data = [
        [[0,0],[0]],
        [[0,1],[1]],
        [[1,0],[1]],
        [[1,1],[0]]
]

# 실행횟수 지정, 학습률 지정, 모멘텀계수 지정
iteration=5000
lr=0.1
mo=0.4

# 활성화 함수 - 시그모이드
# 미분 수행할 때와 아닐 때
def sigmoid(x, derivative=False):
  if(derivative==True):
    return x*(1-x)
  return 1/1(1+np.exp(-x))

# 활성화 함수 - 하이퍼볼릭탄젠트
def tanh(x, derivative=False):
  if(derivative==True):
    return 1-x**2
  return np.tanh(x)

# 가중치 배열선언 함수
def makeMatrix(i, j, fill=0.0):
  mat=[]
  for i in range(i):
    mat.append([fill]*j)
  return mat

# 신경망 정의
class NeuralNetwork:
  #num_x=None
  #num_yh=None
  #num_yo=None
  # 초기화 매서드 (초기값의 지정)
  def __init__(self, num_x, num_yh, num_yo, bias=1):
    # 입력값 num_x, 은닉층초기값 num_yh, 출력층초기값 num_yo, 바이어스
    self.num_x=num_x+bias
    self.num_yh=num_yh
    self.num_yo=num_yo

    # 활성화 함수의 초기값
    self.activation_input=[1.0]*self.num_x
    self.activation_out=[1.0]*self.num_yo
    self.activation_hidden=[1.0]*self.num_yh

    # 가중치 입력 초기값
    self.weight_in=makeMatrix(self.num_x, self.num_yh)
    for i in range(self.num_x):
      for j in range(self.num_yh):
        self.weight_in[i][j]=random.random()

    # 가중치 출력 초기값
    self.weight_out=makeMatrix(self.num_yh, self.num_yo)
    for j in range(self.num_yh):
      for k in range(self.num_yo):
        self.weight_out[j][k]=random.random()

    # 모멘텀 SGD를 위한 이전 가중치 초기값(확률적 경사하강법)
    self.gradient_in=makeMatrix(self.num_x, self.num_yh)
    self.gradient_out=makeMatrix(self.num_yh, self.num_yo)

  # 업데이트 함수
  def update(self, inputs):
    # 입력 레이어의 활성화 함수
    for i in range(self.num_x-1):
      self.activation_input[i]=inputs[i]

    # 은닉층에 대한 활성화 함수 업데이트
    for j in range(self.num_yh):
      sum=0
      for i in range(self.num_x):
        sum=sum+self.weight_in[i][j]*self.activation_input[i]
      # 활성화 함수 지정
      self.activation_hidden[j]=tanh(sum, False)
    
    # 출력층의 활성화 함수 업데이트
    for k in range(self.num_yo):
      sum=0.0
      for j in range(self.num_yh):
        sum=sum+self.weight_out[j][k]*self.activation_hidden[j]
      # 활성화 함수 지정
      self.activation_out[k]=tanh(sum, False)

    return self.activation_out[:]

# -------------------------------------------------------------------

  # 역전파 정의
  def backPropagate(self, targets):
    # 델타 출력 계산
    output_deltas=[0.0]*self.num_yo
    for k in range(self.num_yo):
      error=targets[k] - self.activation_out[k]
      output_deltas[k]=tanh(self.activation_out[k], True)*error

    # 오차 함수
    hidden_deltas=[0.0]*self.num_yh
    for j in range(self.num_yh):
      error=0.0
      for k in range(self.num_yo):
        error=error+self.weight_out[j][k]*output_deltas[k]
        hidden_deltas[j]=tanh(self.activation_hidden[j], True)*error

    # 출력 가중치 업데이트
    for j in range(self.num_yh):
      for k in range(self.num_yo):
        gradient=output_deltas[k]*self.activation_hidden[j]
        v=mo*self.gradient_out[j][k]-lr*gradient
        self.weight_out[j][k]+=v
        self.gradient_out[j][k]=gradient

    # 입력 가중치 업데이트
    for i in range(self.num_x):
      for j in range(self.num_yh):
        gradient=hidden_deltas[j]*self.activation_input[i]
        v=mo*self.gradient_in[i][j]-lr*gradient
        self.weight_in[i][j]+=v
        self.gradient_in[i][j]=gradient

    # 오차 계산
    error=0.0
    for k in range(len(targets)):
      error=error+0.5*(targets[k]-self.activation_out[k])**2
    return error

  # 학습 정의
  def train(self, patterns):
    for i in range(iteration):
      error=0.0
      for p in patterns:
        inputs=p[0]
        targets=p[1]
        self.update(inputs)
        error=error+self.backPropagate(targets)
      if i % 500 == 0:
        print('error : %-.5f' % error)

  # 결과값 정의
  def result(self, patterns):
    for p in patterns:
      print('input : %s, Predict : %s' % (p[0], self.update(p[0])))

if __name__=='__main__':
  # 신경망 호출
  n=NeuralNetwork(2,2,1)

  # 학습실행
  n.train(data)
  # 결과값 출력
  n.result(data)


error : 0.49933
error : 0.00239
error : 0.00084
error : 0.00050
error : 0.00035
error : 0.00027
error : 0.00022
error : 0.00018
error : 0.00016
error : 0.00014
input : [0, 0], Predict : [0.000603614493008657]
input : [0, 1], Predict : [0.9890001944352643]
input : [1, 0], Predict : [0.9890259172139294]
input : [1, 1], Predict : [0.002145608950131172]
